In [177]:
# %load model.py

In [223]:
import numpy as np
#continuous intervals
first = 1
last = 100

In [224]:
def continuous(first, last):
	domain = np.arange(0, 100, 1)
	i = 1
	n = last*(last + 1)/2
	hyps = []
	names = []
	sizes = []
	for s in domain:
		local_domain = np.arange(s, last, 1)
		for e in local_domain:
			hyps.append(np.arange(s, e+1,1))
			names.append(["interval from " + str(s) + " to " + str(e)])
			sizes.append(len(hyps[i-1]))
			i = i + 1
	return [hyps, names, sizes]

In [225]:
# all number
def all_number(first, last):
	all =  np.arange(0, 100, 1)
	names = "all"
	sizes = len(all)
	return [all, names, sizes]

In [226]:
# even or odd
def even_odd(first, last):
	domain = np.arange(0, 100, 1)
	evens = [x for x in domain if x % 2 == 0]
	odds = [x for x in domain if x % 2 != 0]
	hyps = np.array((evens, odds))
	names = ["evens", "odds"]
	sizes = [len(evens), len(odds)]
	return [hyps, names, sizes]

In [227]:
# powers
def powers (first, last):
	domain = np.arange(0, 100, 1)
	power = [2, 3, 4, 5, 6]
	n = len(power)
	hyps = []
	names = []
	sizes = []
	for i in np.arange(1, n, 1):
		p = power[i-1]
		hyps.append([n**p for n in domain if n**p<=last])
		names.append(["powers of " + str(p)])
		sizes.append(len(hyps[i-1]))
	return [hyps, names, sizes]


In [228]:
# a chart of all lower hypothesis
def hypothesis_chart(first, last):
	c=continuous(first, last)
	n=all_number(first, last)
	eo=even_odd(first, last)
	p=powers(first, last)
	return [c, n, eo, p]

In [229]:
# lower hypothesis space
###### mag first, math second ######
c=continuous(first, last)
n=all_number(first, last)
eo=even_odd(first, last)
p=powers(first, last)
nmag = len(c[0]) + len(n[0])
nmath = len(eo[0]) + len(p[0])
hypothesis_space=[]
hypothesis_space.extend(c[0])
hypothesis_space.extend(n[0])
hypothesis_space.extend(eo[0])
hypothesis_space.extend(p[0])

In [230]:
# covert lower hypothesis space to binary
last = 100
nhyp = len(hypothesis_space)
print (nhyp)
hypothesisbin = np.zeros(shape = [last, nhyp])

5156


In [231]:
# overhypothesis space
##### mag first, math second ######
mag_space = np.append(np.repeat(1, nmath), np.repeat(0, nmag))
math_space = np.append(np.repeat(0, nmath), np.repeat(1, nmag))
overhypothesis = np.stack((mag_space, math_space))

In [232]:
# lambda
math_prior = 0.9
# 1-lambda
mag_prior = 1-math_prior

In [233]:
### prior ###

##### mag first, math second ######

# low-level prior
prior_mag = mag_prior/nmag
prior_math = math_prior/nmath
low_prior = []
low_prior.extend(np.repeat(prior_mag, nmag))
low_prior.extend(np.repeat(prior_math, nmath))
low_prior = np.matrix(low_prior)


In [234]:
# theory level prior
##### mag first, math second ######
theory_prior = np.matrix((mag_prior, math_prior))
print (theory_prior.shape)

(1, 2)


In [235]:
# lower-level matrix
domain = np.arange(0, 100, 1)
print (len(domain))
lower_level_matrix = np.empty(shape=[0, nhyp])
for d in domain:
	d_binary = []
	for h in hypothesis_space:
		if np.in1d(d, h)[0]:
			d_binary.append(1)
		else:
			d_binary.append(0)
	lower_level_matrix = np.append(lower_level_matrix, [d_binary], axis = 0)
print (lower_level_matrix)

100
[[ 1.  1.  1. ...,  1.  1.  1.]
 [ 0.  1.  1. ...,  1.  1.  1.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [236]:
# lower_level likelihood
lower_level_likelihood_deno = np.sum(lower_level_matrix, axis = 0)
lower_level_likelihood = np.matrix([1/p for p in lower_level_likelihood_deno])

In [237]:
# lower_level likelihood of an observation
observation = 2
lower_level_likelihood_observation = np.multiply(lower_level_matrix[observation,:], lower_level_likelihood)

In [238]:
### Step 1 - theory-level likelihood ###
theory_level_likelihood_mag=np.multiply(np.matrix(overhypothesis[0,:]).transpose(),low_prior.transpose())
theory_level_likelihood_math=np.multiply(np.matrix(overhypothesis[1,:]).transpose(),low_prior.transpose())
theory_level_likelihood = np.stack((theory_level_likelihood_mag.transpose(), theory_level_likelihood_math.transpose()))
print (theory_level_likelihood.shape)
    

(2, 5156)


In [239]:
### Step 2 ###
transpose_lower_likeli = lower_level_likelihood_observation.transpose()
#print (transpose_lower_likeli.shape)
# prior for observation
#prob_observation_theory = transpose_lower_likeli*theory_level_likelihood


prob_observation_theory = []
for row in theory_level_likelihood:
    a = np.multiply(row, lower_level_likelihood_observation)
    prob_observation_theory.append(np.sum(a))
prob_observation_theory = np.matrix(prob_observation_theory)
print (prob_observation_theory.shape)

(1, 2)


In [244]:
### Step 3 theory-level posterior distribution ###
numerator = np.multiply(prob_observation_theory, theory_prior)
denominator = np.sum(numerator)
theory_posterior_mag = numerator[0,0]/denominator
theory_posterior_num = numerator[0,1]/denominator
theory_level_posterior = np.matrix([theory_posterior_mag, theory_posterior_num])
print (np.sum(theory_level_posterior))

1.0


In [247]:
### Step 3a lower-level posterior distribution ###
numerator = np.multiply(low_prior, lower_level_likelihood_observation)
denominator = np.sum(numerator)
i = 0
lower_level_posterior = []
while i < nhyp:
    lower_level_posterior.append(numerator[0,i]/denominator)
    i = i +1
lower_level_posterior = np.matrix(lower_level_posterior)
print (np.sum(lower_level_posterior))

1.0


In [242]:
### Step 4 re-assigning priors ###
math_prior = theory_level_posterior[0,1]
mag_prior = 1 - math_prior
# low level #
denominator = np.sum(lower_level_posterior)
numerator = []
i = 0
while i < nmag:
	numerator.append(mag_prior*lower_level_posterior[0, i])
	i = i + 1
m = nmag
while m < (nhyp):
	numerator.append(math_prior * lower_level_posterior[0, m])
	m = m + 1
    
lower_level_new_prior = np.matrix([n/denominator for n in numerator])

In [243]:
# posterior predictive


In [ ]:
# assign theory level prior


In [ ]:
# theory-level prior --- depend on initial assignment and last observered number
# low-level prior --- depend on theory level prior
# lower_level_observation_likelihood remains constant
# theory_level_likelihood --- depend on low_level_prior, which depend on theory level prior, which depend on last observred number
# prob_observation_theor --- depend on theory_level_likelihood
# theory_level_posterior --- theory_level_prior, theory_level_likelihood(low_level_prior, )
# 

In [111]:
# matrix multiplication
a = [[2, 2, 2],[3, 3, 3]] # 2*3
b = [[1], [2], [3]] #3*1
c = np.matrix(a)*np.matrix(b)
print (c)

# element wise product
m = [1, 2, 3, 4, 5]
n = [4, 5, 6, 7, 8]
p = np.multiply(np.array(m), np.array(n))
print (p)

[[12]
 [18]]
[ 4 10 18 28 40]
